Start with a goal  
* Want to know where we can go camping near Mount Hood
* Want to know how many sites are at a campground
* Want to know what sites (number of sites?) are accessible, near water, toilet y/n, and allow pets


What are some camping analogies that might relate to what Im trying to convey about scalability?
* Send friends to multiple campgrounds to help find sites instead of everyone going to the same place - parallelization
* Come back to rendevous point because no cell service and compare what we found - 5 sites near water, no accesable sites, 

In [102]:
import json
import pandas as pd
import itertools
import requests

import config

Getting a list of facilities from RIDB
lat/long  
or state as comma delimited list of 2 char state codes  
https://ridb.recreation.gov/docs  
https://ridb.recreation.gov/ridb-access-agreement 

In [13]:
ridb_facilities_url = "https://ridb.recreation.gov/api/v1/facilities"

In [18]:
config.RIDB_API_KEY

'b2b7cf12-0a8a-413b-aa75-3125b41977b8'

In [45]:

params = {"activity_id":9, "latitude":45.4977712, "longitude":-121.8211673, "radius":15}
headers = {"accept": "application/json", "apikey": "b2b7cf12-0a8a-413b-aa75-3125b41977b8"}
response = requests.get(ridb_facilities_url, params, headers=headers)
camping_json  = json.loads(response.text)

In [46]:
response.status_code

200

In [47]:
camping_json['RECDATA'][0]

{'FacilityID': '232834',
 'LegacyFacilityID': '71617',
 'OrgFacilityID': 'AN371617',
 'ParentOrgID': '131',
 'ParentRecAreaID': '1106',
 'FacilityName': 'RILEY HORSE CAMPGROUND',
 'FacilityDescription': "<h2>Overview</h2>\nRiley Campground is an equestrian facility located northwest of Oregon's majestic Mt. Hood. Horse corrals are tucked into the forest near the campsites, and several horseback riding trails can be accessed from the campground. A variety of recreational and sightseeing opportunities are available for visitors to enjoy.<h2>Recreation</h2>\nSeveral hiking, biking and horseback riding trails can be accessed from the campground, including the 5.3-mile Cast Creek Trail, which was reconstructed in 1998 and leads from the campground to the Zigzag Mountain Trail. Combined with the nearby 4.9-mile Horseshoe Ridge Trail and part of the Zigzag Mountain Trail, visitors can make a nearly 14-mile excursion. The Sandy River Trail follows the river and stretches about 3 miles.\n<br />

In [76]:
df = pd.DataFrame(camping_json['RECDATA'])

In [77]:
df.columns

Index(['FacilityID', 'LegacyFacilityID', 'OrgFacilityID', 'ParentOrgID',
       'ParentRecAreaID', 'FacilityName', 'FacilityDescription',
       'FacilityTypeDescription', 'FacilityUseFeeDescription',
       'FacilityDirections', 'FacilityPhone', 'FacilityEmail',
       'FacilityReservationURL', 'FacilityMapURL', 'FacilityAdaAccess',
       'GEOJSON', 'FacilityLongitude', 'FacilityLatitude', 'Keywords',
       'StayLimit', 'Reservable', 'Enabled', 'LastUpdatedDate'],
      dtype='object')

In [78]:
df.iloc[0]['GEOJSON']

{'TYPE': 'Point', 'COORDINATES': [-121.8594444, 45.3813889]}

In [79]:
df.iloc[0][['FacilityLongitude', 'FacilityLatitude']]

FacilityLongitude   -121.859444
FacilityLatitude      45.381389
Name: 0, dtype: object

For each facility we want to get campground information

In [80]:
# Hmm! Not all these facilities are campgrounds

In [81]:
df[['FacilityID', 'FacilityName']]

,FacilityID,FacilityName
0,232834,RILEY HORSE CAMPGROUND
1,234075,WILDWOOD RECREATION SITE
2,122940,WYETH CAMPGROUND
3,10070260,Vista Ridge Trailhead
4,234306,EAGLE CREEK OVERLOOK GRP SITE
5,251434,LOST LAKE RESORT AND CAMPGROUND
6,272093,SHERWOOD CAMPGROUND
7,233329,TILLY JANE GUARD STATION
8,232836,TOLLGATE
9,232835,STILL CREEK


In [82]:
# Get campsite data for each area, if no campsites then drop --- or should we?
# Consider how we would scale this out - 
# Lost Lake example

In [58]:
resp = requests.get(f"{ridb_facilities_url}/251434/campsites", headers=headers)

In [59]:
resp.status_code

200

In [69]:
campsites = json.loads(resp.text)

In [72]:
len(campsites['RECDATA'])

50

In [83]:
df_campsites = pd.DataFrame(campsites['RECDATA'])

In [84]:
df_campsites

,CampsiteID,FacilityID,CampsiteName,CampsiteType,TypeOfUse,Loop,CampsiteAccessible,CampsiteLongitude,CampsiteLatitude,CreatedDate,LastUpdatedDate,ATTRIBUTES,ENTITYMEDIA,PERMITTEDEQUIPMENT
0,96115,251434,A021,RV NONELECTRIC,Overnight,A Loop,False,-121.816561,45.498111,2015-04-09,2020-10-15,"[{'AttributeName': 'Driveway Grade', 'Attribut...",[{'EntityMediaID': '2cf26ca1-bbdd-4619-99ff-47...,"[{'EquipmentName': 'Trailer', 'MaxLength': 81}..."
1,96043,251434,C005,RV NONELECTRIC,Overnight,C Loop,True,-121.813871,45.496703,2015-04-09,2020-10-15,"[{'AttributeName': 'Driveway Entry', 'Attribut...",[{'EntityMediaID': '06a441c6-41be-408a-99bb-71...,"[{'EquipmentName': 'Trailer', 'MaxLength': 54}..."
2,96075,251434,D001,RV NONELECTRIC,Overnight,D Loop,True,-121.815180,45.494484,2015-04-09,2020-10-15,"[{'AttributeName': 'Accessibility', 'Attribute...",[{'EntityMediaID': '7c833b0a-b0cd-4093-8c91-95...,"[{'EquipmentName': 'Trailer', 'MaxLength': 36}..."
3,96104,251434,A009,RV NONELECTRIC,Overnight,A Loop,False,-121.816130,45.498334,2015-04-09,2020-10-15,"[{'AttributeName': 'Base Number of People', 'A...",[{'EntityMediaID': '9582815e-0d79-41c5-aa12-0e...,"[{'EquipmentName': 'Trailer', 'MaxLength': 34}..."
4,96032,251434,H001,EQUESTRIAN NONELECTRIC,Overnight,Horsecamp,True,-121.811069,45.484083,2015-04-09,2020-10-15,"[{'AttributeName': 'Horse Stall/Corral', 'Attr...",[{'EntityMediaID': '2590bc58-4289-4a98-934a-6f...,"[{'EquipmentName': 'Horse', 'MaxLength': 32}]"
5,96014,251434,B012,RV NONELECTRIC,Overnight,B Loop,False,-121.815563,45.497653,2015-04-09,2020-10-15,"[{'AttributeName': 'Placed on Map', 'Attribute...",[{'EntityMediaID': 'b4ce9a7d-cea7-421e-a7f7-38...,"[{'EquipmentName': 'Trailer', 'MaxLength': 100..."
6,96127,251434,B010,RV NONELECTRIC,Overnight,B Loop,True,-121.814879,45.498148,2015-04-09,2020-10-15,"[{'AttributeName': 'Base Number of Vehicles', ...",[{'EntityMediaID': 'f9c7eb69-43ba-4481-835a-dd...,"[{'EquipmentName': 'Trailer', 'MaxLength': 42}..."
7,96083,251434,A017,RV NONELECTRIC,Overnight,A Loop,True,-121.816303,45.499002,2015-04-09,2020-10-15,"[{'AttributeName': 'Picnic Table', 'AttributeV...",[{'EntityMediaID': 'b1f38ecf-6a27-45ce-a6c3-15...,"[{'EquipmentName': 'Trailer', 'MaxLength': 33}..."
8,96023,251434,B004,RV NONELECTRIC,Overnight,B Loop,False,-121.814691,45.497121,2015-04-09,2020-10-15,"[{'AttributeName': 'Grills/Fire Ring', 'Attrib...",[{'EntityMediaID': '37449323-49fb-4bdc-ac6a-fe...,"[{'EquipmentName': 'Trailer', 'MaxLength': 33}..."
9,96040,251434,D012,TENT ONLY NONELECTRIC,Overnight,D Loop,False,-121.815892,45.492929,2015-04-09,2020-10-15,"[{'AttributeName': 'Shade', 'AttributeValue': ...",[{'EntityMediaID': 'c37a1637-e5d3-406b-be7c-cd...,"[{'EquipmentName': 'Tent', 'MaxLength': 15}]"


50 campsites, compare with the lostlake website that says 125:  
https://www.fs.usda.gov/recarea/mthood/recarea/?recid=53228 45.48889, -121.82194

In [ ]:
[entry['AttributeName'] for entry in df_campsites.iloc[0].ATTRIBUTES]

In [103]:
set(itertools.chain(*df_campsites['ATTRIBUTES'].apply(lambda x: [entry['AttributeName'] for entry in x])))

{'Accessibility',
 'BBQ',
 'Base Number of People',
 'Base Number of Vehicles',
 'Campfire Allowed',
 'Capacity/Size Rating',
 'Checkin Time',
 'Checkout Time',
 'Double Driveway',
 'Driveway Entry',
 'Driveway Grade',
 'Driveway Length',
 'Driveway Surface',
 'Fire Pit',
 'Grills/Fire Ring',
 'Hike In Distance to Site',
 'Horse Hitching Post',
 'Horse Stall/Corral',
 'IS EQUIPMENT MANDATORY',
 'Location Rating',
 'Max Num of People',
 'Max Num of Vehicles',
 'Max Vehicle Length',
 'Min Num of People',
 'Pets Allowed',
 'Picnic Table',
 'Placed on Map',
 'Platform',
 'Privacy',
 'Proximity to Water',
 'Quiet Area',
 'Shade',
 'Site Access',
 'Site Height/Overhead Clearance',
 'Site Rating',
 'Tent Pad',
 'Tent Pad Length',
 'Tent Pad Width'}

In [86]:
df.query("FacilityID == '251434'")

,FacilityID,LegacyFacilityID,OrgFacilityID,ParentOrgID,ParentRecAreaID,FacilityName,FacilityDescription,FacilityTypeDescription,FacilityUseFeeDescription,FacilityDirections,...,FacilityMapURL,FacilityAdaAccess,GEOJSON,FacilityLongitude,FacilityLatitude,Keywords,StayLimit,Reservable,Enabled,LastUpdatedDate
5,251434,125541,AN425541,131,1106,LOST LAKE RESORT AND CAMPGROUND,<h2>Overview</h2>\nLost Lake Campground is cou...,Campground,,Getting There:<br /> \nLost Lake Resort &amp; ...,...,,N,"{'TYPE': 'Point', 'COORDINATES': [-121.8219444...",-121.821944,45.488889,,,True,True,2021-03-29
